# Installing packages

In [ ]:
!pip install -q openai --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


# Importing modules

In [ ]:
from openai import OpenAI
import time
import pandas as pd
import os
import json
from google.colab import drive
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="API_KEY",
)

# Google Mounting

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
primate_input_df = pd.read_csv('/content/drive/MyDrive/primate_data.csv')

In [ ]:
primate_input_df.head()

,post_title,post_text,Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down,Feeling-down-depressed-or-hopeless,Feeling-tired-or-having-little-energy,Little-interest-or-pleasure-in-doing,Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual,Poor-appetite-or-overeating,Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way,Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television,Trouble-falling-or-staying-asleep-or-sleeping-too-much
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",1,0.0,1,1,0,0.0,0.0,0.0,0
1,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,1,1.0,0,0,1,0.0,0.0,0.0,0
2,Stuck between moving forward and killing myself,Some background information: My GF of almost 3...,1,1.0,1,1,0,0.0,1.0,0.0,0
3,I need help on how to help my girlfriend.,"My girlfriend ,of about 3 months now ,has been...",1,1.0,0,0,0,0.0,1.0,0.0,0
4,I've been feeling this way for some tine,I'm alway feeling like this. It doesn't even m...,1,1.0,0,0,0,0.0,0.0,0.0,0


# Task
Note: Not all the cells must be runned!

In [ ]:
pre_processed_labels = {
    "Q1": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
    "Q2": "Feeling down depressed or hopeless",
    "Q3": "Feeling tired or having little energy",
    "Q4": "Little interest or pleasure in doing",
    "Q5": "Moving or speaking so slowly that other people could have noticed Or the opposite being so fidgety or restless that you have been moving around a lot more than usual",
    "Q6": "Poor appetite or overeating",
    "Q7": "Thoughts that you would be better off dead or of hurting yourself in some way",
    "Q8": "Trouble concentrating on things such as reading the newspaper or watching television",
    "Q9": "Trouble falling or staying asleep or sleeping too much"

}


In [ ]:
def get_answer(post_title, post_text, label):
  try:
    messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]
    intro_message = f"""
    I want to collect a dataset for my machine learning task which is related to mental health issues. \
    I have found the PRIMATE dataset which was collected in 2022. \
    The dataset contains 2003 records, of which every record has the below structure: \
      post_title - post_text - Q1 - Q2 - Q3 - Q4 - Q5 - Q6 - Q7 - Q8 - Q9 \
      The post_titles, and post_texts were collected from the Reddit website, \
      and the Q1 to Q9 were some matrices based on the PHQ-9(Patient Health Questionnaire ). \
      The value of the Q1 to Q9 questions can be deduced/known using the content present in the post text and its title. \
      The matrices in order are: \
      1. Feeling bad about yourself or that you are a failure or have let yourself or your family down \
      2. Feeling down depressed or hopeless \
      3. Feeling tired or having little energy \
      4. Little interest or pleasure in doing \
      5. Moving or speaking so slowly that other people could have noticed Or the opposite being so fidgety or restless that you have been moving around a lot more than usual \
      6. Poor appetite or overeating \
      7. Thoughts that you would be better off dead or of hurting yourself in some way \
      8. Trouble concentrating on things such as reading the newspaper or watching television \
      9. Trouble falling or staying asleep or sleeping too much \

      I will give the post title with its post text and a label which are delimited by triple backticks. \
      But I want from you to tell me whether the given label(question) would be yes or no by your own inferring. \
      For inferring, and generating the appropriate reason please also take a glance through post title. \
      If the post text does not contain enought information about one of this metrices just label it with "no". \
      If the post text or post title contains some ilegal problems please ignore them as you can. \
      Since I am just trying to compare your outputs with the output of downloaded dataset. \
      Provide your answer for the given data in JSON format with the following keys: \
      label, value, post_title, reason.

      This is the given post title: ```{post_title}```, this is the post text: ```{post_text}```, and this is the label: ```{label}```.
    """
    messages.append( {"role": "user", "content": intro_message}, )
    chat = client.chat.completions.create( model="gpt-3.5-turbo-0613", messages=messages )
    reply = chat.choices[0].message.content
    return True, reply

  except Exception as e:
    return False, e

In [ ]:
def get_answer_2305_08377(post_title, post_text, label):
  try:
    messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Hello, assistant."},
                {"role": "assistant", "content": "Hello! How can I assist you today?"},
                {"role": "user", "content": "Classify this post for the given post label. Your answer must be only a JSON object with no further explanations."}
              ]
    intro_message = f"""
    This is a text classification task.
    I have collected a dataset for my machine learning task which is related to mental health issues. \
    I have found the PRIMATE dataset which was collected in 2022. \
    The dataset contains 2003 records, of which every record has the below structure: \
      post_title - post_text - Q1 - Q2 - Q3 - Q4 - Q5 - Q6 - Q7 - Q8 - Q9 \
      The post_titles, and post_texts were collected from the Reddit website, \
      and the Q1 to Q9 were some matrices based on the PHQ-9(Patient Health Questionnaire ). \
      The value of the Q1 to Q9 questions can be deduced/known using the content present in the post text and its title. \
      The matrices in order are: \
      1. Feeling bad about yourself or that you are a failure or have let yourself or your family down \
      2. Feeling down depressed or hopeless \
      3. Feeling tired or having little energy \
      4. Little interest or pleasure in doing \
      5. Moving or speaking so slowly that other people could have noticed Or the opposite being so fidgety or restless that you have been moving around a lot more than usual \
      6. Poor appetite or overeating \
      7. Thoughts that you would be better off dead or of hurting yourself in some way \
      8. Trouble concentrating on things such as reading the newspaper or watching television \
      9. Trouble falling or staying asleep or sleeping too much \

      I will give the post title with its post text and a label which are delimited by triple backticks. \
      But I want from you to tell me whether the given label(question) would be yes or no by your own inferring. \

      First, detect CLUES (i.e., keywords, phrases, contextual informations) that support the label determination of input.
      Second, deduce the diagnostic REASONING process from premises (i.e., clues, post text, post title, label) that supports the INPUT
      label determination (Limit the number of words to 100).
      Third, based on clues, reasoning and input, determine whether the given label(question) would be yes or no for the given inputs.
      If the post text or post title contains some ilegal problems please ignore them as you can. \
      Since I am just trying to compare your outputs with the output of downloaded dataset. \
      Provide your answer for the given data in JSON format with the following keys: \
      label, value(yes/no), post_title, CLUEs.

      This is the given post title: ```{post_title}```, this is the post text: ```{post_text}```, and this is the label: ```{label}```.
    """
    messages.append( {"role": "user", "content": intro_message}, )
    chat = client.chat.completions.create( model="gpt-3.5-turbo-0613", messages=messages )
    reply = chat.choices[0].message.content
    return True, reply

  except Exception as e:
    return False, e

Only for one sample, and one label

In [ ]:
first_row = df.iloc[0]
boolean, gpt_answer = get_answer_2305_08377(first_row['post_title'], first_row['post_text'], pre_processed_labels[0])
print(f"The result is: {boolean} and {gpt_answer}")
if boolean:
  load_gpt_ans_to_json = json.loads(gpt_answer)
  print(load_gpt_ans_to_json)
  # dump_gpt_ans = json.dumps(load_gpt_ans, indent=2)
  # print(dump_gpt_ans)
  result = {
      "bool": boolean,
      "json": load_gpt_ans_to_json
  }

if not boolean:
  result = {
      "bool": boolean,
      "error": gpt_answer
  }
# Create a new JSON file for each iteration
file_path = f"gpt_result_1:21.json"  # Customize the file name as desired

with open(file_path, "w") as json_file:
    json.dump(result, json_file, indent=2)

The result is: True and {
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
  "value": "yes",
  "post_title": "I don't feel original anymore.",
  "CLUEs": ["high school band", "highest competitors", "straight academics", "exercised daily", "enlisted in the military", "worst decisions", "motivated", "patriot", "set myself up for success", "set myself up financially", "no happiness", "no honor", "not progressing anywhere", "replaceable", "going back home", "closer to my family", "nothing here satisfies me", "tried everything to be happy", "wish somebody could help"]
}
{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': "I don't feel original anymore.", 'CLUEs': ['high school band', 'highest competitors', 'straight academics', 'exercised daily', 'enlisted in the military', 'worst decisions', 'motivated', 'patriot', 'set myself up for success', 'set

## First 10 rows

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
first_ten_df = pd.DataFrame(columns=columns)

In [ ]:
print(first_ten_df)

Empty DataFrame
Columns: [post_title, label, value]
Index: []


In [ ]:
first_ten_rows =  primate_input_df.iloc[6:10]

total_rows = len(first_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in first_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, value)
      print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        load_gpt_ans_to_json = json.loads(gpt_answer)
        print(load_gpt_ans_to_json)
        # dump_gpt_ans = json.dumps(load_gpt_ans, indent=2)
        # print(dump_gpt_ans)
        result = {
            "bool": boolean,
            "json": load_gpt_ans_to_json
        }

        first_ten_df = first_ten_df.append({
            'post_title': load_gpt_ans_to_json['post_title'],
            'label': load_gpt_ans_to_json['label'],
            'value': load_gpt_ans_to_json['value']
        }, ignore_index=True)
      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)


Processing Rows:   0%|          | 0/36 [00:09<?, ?it/s]
<ipython-input-13-bd1d209dfd18>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_ten_df = first_ten_df.append({
Processing Rows:   3%|▎         | 1/36 [00:14<08:21, 14.33s/it]

The result is: True and {
  "label": "Feeling bad about yourself or that you are a failure or have let yourself or your family down",
  "value": "yes",
  "post_title": "This is me finally talking about “it”.",
  "CLUEs": [
    "wish he could open up and tell me how he’s feeling",
    "wish he could cry with me",
    "We make love and make memories and we shine",
    "depression isn’t something that can just be fixed"
  ]
}
{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': 'This is me finally talking about “it”.', 'CLUEs': ['wish he could open up and tell me how he’s feeling', 'wish he could cry with me', 'We make love and make memories and we shine', 'depression isn’t something that can just be fixed']}


<ipython-input-13-bd1d209dfd18>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_ten_df = first_ten_df.append({
Processing Rows:   6%|▌         | 2/36 [00:21<05:36,  9.91s/it]

The result is: True and {
  "label": "Feeling down depressed or hopeless",
  "value": "Yes",
  "post_title": "This is me finally talking about \"it\".",
  "CLUEs": [
    "dark days",
    "mood is self-described as \"blah\"",
    "not feeling well",
    "house feels tight",
    "fight them"
  ]
}
{'label': 'Feeling down depressed or hopeless', 'value': 'Yes', 'post_title': 'This is me finally talking about "it".', 'CLUEs': ['dark days', 'mood is self-described as "blah"', 'not feeling well', 'house feels tight', 'fight them']}


<ipython-input-13-bd1d209dfd18>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_ten_df = first_ten_df.append({
Processing Rows:   8%|▊         | 3/36 [00:32<05:56, 10.80s/it]

...

The result is: True and {
  "label": "Thoughts that you would be better off dead or of hurting yourself in some way",
  "value": "no",
  "post_title": "Being the boyfriend.",
  "CLUEs": ["suicidal thoughts occasionally", "depression", "held a gun to her head", "her serious depression", "she told me she wants to die", "scared for her", "relationship is about her", "she has enough to worry about"]
}
{'label': 'Thoughts that you would be better off dead or of hurting yourself in some way', 'value': 'no', 'post_title': 'Being the boyfriend.', 'CLUEs': ['suicidal thoughts occasionally', 'depression', 'held a gun to her head', 'her serious depression', 'she told me she wants to die', 'scared for her', 'relationship is about her', 'she has enough to worry about']}


<ipython-input-13-bd1d209dfd18>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_ten_df = first_ten_df.append({
Processing Rows:  97%|█████████▋| 35/36 [05:17<00:10, 10.20s/it]

The result is: True and {"label": "Trouble concentrating on things such as reading the newspaper or watching television", "value": "yes", "post_title": "Being the boyfriend.", "CLUEs": ["relationship has dwindled", "don't have conversations like we used to", "she always falls asleep on the couch so we don't sleep together hardly", "don't have sex at all", "she's not in the mood", "paranoid that she's cheating on me sometimes", "entire relationship is about her", "constantly having internal debate"]}
{'label': 'Trouble concentrating on things such as reading the newspaper or watching television', 'value': 'yes', 'post_title': 'Being the boyfriend.', 'CLUEs': ['relationship has dwindled', "don't have conversations like we used to", "she always falls asleep on the couch so we don't sleep together hardly", "don't have sex at all", "she's not in the mood", "paranoid that she's cheating on me sometimes", 'entire relationship is about her', 'constantly having internal debate']}


<ipython-input-13-bd1d209dfd18>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_ten_df = first_ten_df.append({
Processing Rows: 100%|██████████| 36/36 [05:25<00:00,  9.54s/it]

The result is: True and {
  "label": "Trouble falling or staying asleep or sleeping too much",
  "value": "yes",
  "post_title": "Being the boyfriend.",
  "CLUEs": ["alone doing laundry", "hysterically crying", "Earlier that day she had called me crying", "she just wants to give up", "She wants to die", "scared for her and for us", "Am I just a shitty person for wanting it to be about me sometimes?", "I want our relationship to be what it once was; fun and exciting"]
}
{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'yes', 'post_title': 'Being the boyfriend.', 'CLUEs': ['alone doing laundry', 'hysterically crying', 'Earlier that day she had called me crying', 'she just wants to give up', 'She wants to die', 'scared for her and for us', 'Am I just a shitty person for wanting it to be about me sometimes?', 'I want our relationship to be what it once was; fun and exciting']}


In [ ]:
first_ten_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  96 non-null     object
 1   label       96 non-null     object
 2   value       96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [ ]:
first_ten_df.to_csv('first_ten_output.csv', index=False)

## Second 10 rows

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
seond_ten_df = pd.DataFrame(columns=columns)

In [ ]:
second_ten_rows =  primate_input_df.iloc[16:20]

total_rows = len(second_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in second_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, value)
      # print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        load_gpt_ans_to_json = json.loads(gpt_answer)
        print(load_gpt_ans_to_json)
        result = {
            "bool": boolean,
            "json": load_gpt_ans_to_json
        }

        seond_ten_df = seond_ten_df.append({
            'post_title': load_gpt_ans_to_json['post_title'],
            'label': load_gpt_ans_to_json['label'],
            'value': load_gpt_ans_to_json['value']
        }, ignore_index=True)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)

Processing Rows:   0%|          | 0/36 [00:29<?, ?it/s]
<ipython-input-48-fc00489da79d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  seond_ten_df = seond_ten_df.append({
Processing Rows:   3%|▎         | 1/36 [00:09<05:44,  9.84s/it]

{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': 'Is it okay to drop a class for the sake of my own health?', 'CLUEs': ['stressing me out', "can't handle the workload", 'advice before deciding something important']}


<ipython-input-48-fc00489da79d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  seond_ten_df = seond_ten_df.append({
Processing Rows:   6%|▌         | 2/36 [00:21<06:12, 10.94s/it]

{'label': 'Feeling down depressed or hopeless', 'value': 'yes', 'post_title': 'Is it okay to drop a class for the sake of my own health?', 'CLUEs': ['stressed out', 'handle the workload', 'relying on advice', 'feeling down', 'hopeful for advice']}


<ipython-input-48-fc00489da79d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  seond_ten_df = seond_ten_df.append({
Processing Rows:   8%|▊         | 3/36 [00:33<06:16, 11.41s/it]

...

{'label': 'Thoughts that you would be better off dead or of hurting yourself in some way', 'value': 'yes', 'post_title': 'I dont know what to do anymore', 'CLUEs': ['suicide', 'worthless', 'no one likes me', 'beat me in the past', 'pussy to kill myself', 'lifting words']}


<ipython-input-48-fc00489da79d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  seond_ten_df = seond_ten_df.append({
Processing Rows:  97%|█████████▋| 35/36 [05:41<00:09,  9.72s/it]

{'label': 'Trouble concentrating on things such as reading the newspaper or watching television', 'value': 'yes', 'post_title': 'I dont know what to do anymore', 'CLUEs': ['cant make friends', 'antisocial', 'depressed']}


<ipython-input-48-fc00489da79d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  seond_ten_df = seond_ten_df.append({
Processing Rows: 100%|██████████| 36/36 [05:47<00:00,  8.64s/it]

{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'yes', 'post_title': 'I dont know what to do anymore', 'CLUEs': ['cant make friends', 'depressed', 'feeling bad', 'not willing to do anything', 'trouble sleeping']}


In [ ]:
seond_ten_df.info()
seond_ten_df.to_csv('seond_ten_output.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  87 non-null     object
 1   label       87 non-null     object
 2   value       87 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


## Third 10 rows

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
third_ten_df = pd.DataFrame(columns=columns)

In [ ]:
third_ten_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  60 non-null     object
 1   label       60 non-null     object
 2   value       60 non-null     object
dtypes: object(3)
memory usage: 1.5+ KB


In [ ]:
third_ten_rows =  primate_input_df.iloc[28:30]

total_rows = len(third_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in third_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, value)
      # print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        load_gpt_ans_to_json = json.loads(gpt_answer)
        print(load_gpt_ans_to_json)
        result = {
            "bool": boolean,
            "json": load_gpt_ans_to_json
        }

        third_ten_df = third_ten_df.append({
            'post_title': load_gpt_ans_to_json['post_title'],
            'label': load_gpt_ans_to_json['label'],
            'value': load_gpt_ans_to_json['value']
        }, ignore_index=True)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)

Processing Rows:   0%|          | 0/18 [24:09<?, ?it/s]
<ipython-input-17-a76b9f0d44a1>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_ten_df = third_ten_df.append({
Processing Rows:   6%|▌         | 1/18 [00:20<05:40, 20.04s/it]

{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': 'So Much Pain In The World', 'CLUEs': ['struggling with suicidal thoughts', 'suffering I see in the world', 'how someone can have 4 houses', 'take 12 cruises a year', 'fly around in private jets', 'selfish as a species', 'Darwinism no longer controls', 'survival of the wealthy']}


<ipython-input-17-a76b9f0d44a1>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_ten_df = third_ten_df.append({
Processing Rows:  11%|█         | 2/18 [00:28<03:35, 13.48s/it]

{'label': 'Feeling down depressed or hopeless', 'value': 'yes', 'post_title': 'So Much Pain In The World', 'CLUEs': ['suicide note', 'struggling with suicidal thoughts', 'suffering', 'selfish', 'hopelessness']}


<ipython-input-17-a76b9f0d44a1>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_ten_df = third_ten_df.append({
Processing Rows:  17%|█▋        | 3/18 [00:39<03:00, 12.04s/it]

...

{'label': 'Thoughts that you would be better off dead or of hurting yourself in some way', 'value': 'yes', 'post_title': "am I suicidal even if I know I won't kill myself?", 'CLUEs': ['nice it would be to be dead', 'seriously considered self-harm', 'strong urge to do so', 'just want everything to end', 'no urge to change anything', 'considered suicidal']}


<ipython-input-17-a76b9f0d44a1>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_ten_df = third_ten_df.append({
Processing Rows:  94%|█████████▍| 17/18 [13:50<00:18, 18.08s/it]

{'label': 'Trouble concentrating on things such as reading the newspaper or watching television', 'value': 'No', 'post_title': "am I suicidal even if I know I won't kill myself?", 'CLUEs': ['considered self harm', 'escape the internal pain', 'strong urge to do so', 'no drive to do anything', 'irritable as fuck']}


<ipython-input-17-a76b9f0d44a1>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_ten_df = third_ten_df.append({
Processing Rows: 100%|██████████| 18/18 [14:04<00:00, 16.77s/it]

{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'Yes', 'post_title': "am I suicidal even if I know I won't kill myself?", 'CLUEs': ['considered self harm', 'strong urge to do so', 'just want everything to end', 'no drive to do anything', 'irritable as fuck', 'tried to look for psychiatrists and psychologists']}


In [ ]:
third_ten_df.info()
third_ten_df.to_csv('third_ten_output.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  78 non-null     object
 1   label       78 non-null     object
 2   value       78 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


## Forth 10 rows

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
forth_ten_df = pd.DataFrame(columns=columns)

In [ ]:
forth_ten_rows =  primate_input_df.iloc[30:40]

total_rows = len(forth_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in forth_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, value)
      # print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        load_gpt_ans_to_json = json.loads(gpt_answer)
        print(load_gpt_ans_to_json)
        result = {
            "bool": boolean,
            "json": load_gpt_ans_to_json
        }

        forth_ten_df = forth_ten_df.append({
            'post_title': load_gpt_ans_to_json['post_title'],
            'label': load_gpt_ans_to_json['label'],
            'value': load_gpt_ans_to_json['value']
        }, ignore_index=True)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)

Processing Rows:   0%|          | 0/90 [03:41<?, ?it/s]
<ipython-input-21-03a526c5feec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forth_ten_df = forth_ten_df.append({
Processing Rows:   1%|          | 1/90 [00:14<20:51, 14.06s/it]

{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': 'i have been diagnosed with depression but my dad refuses to consider medication', 'CLUEs': ['diagnosed with major depressive disorder', 'dad refuses to consider medication', "believes I'm exaggerating my symptoms", 'struggle doing things I used to love or any basic task']}


<ipython-input-21-03a526c5feec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forth_ten_df = forth_ten_df.append({
Processing Rows:   2%|▏         | 2/90 [00:29<21:25, 14.60s/it]

{'label': 'Feeling down depressed or hopeless', 'value': 'yes', 'post_title': 'i have been diagnosed with depression but my dad refuses to consider medication', 'CLUEs': ['diagnosed', 'depression', 'dad refuses', 'consider medication', 'major depressive disorder', 'antidepressants', 'help', 'exaggerating symptoms', 'trying to be depressed', 'struggle doing things', 'basic task']}


<ipython-input-21-03a526c5feec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forth_ten_df = forth_ten_df.append({
Processing Rows:   3%|▎         | 3/90 [00:42<20:09, 13.91s/it]

...

{'label': 'Thoughts that you would be better off dead or of hurting yourself in some way', 'value': 'Yes', 'post_title': 'HS ruined my life', 'CLUEs': ['ruined my life', 'problems to my mental health', 'stress', 'anxiety', 'social problems', 'bad grades', 'party', 'bullies', 'bully episode', 'harrassed', 'teachers were bad', 'changed classmates', 'find it hard to even study', 'failed my mom', 'situation where I cry a lot', 'I am a failure', 'embarrassing', "you're too young", 'depression', 'mental breakdown', 'force myself into doing things', 'end it right now', 'disappoint to them', 'they know nothing about my mental state']}


<ipython-input-21-03a526c5feec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forth_ten_df = forth_ten_df.append({
Processing Rows:  99%|█████████▉| 89/90 [1:16:35<01:58, 118.65s/it]

{'label': 'Trouble concentrating on things such as reading the newspaper or watching television', 'value': 'yes', 'post_title': 'HS ruined my life', 'CLUEs': ['failed', 'hard to even study', 'struggle when I want to force myself into doing things']}


<ipython-input-21-03a526c5feec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  forth_ten_df = forth_ten_df.append({
Processing Rows: 100%|██████████| 90/90 [1:16:45<00:00, 85.99s/it] 

{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'yes', 'post_title': 'HS ruined my life', 'CLUEs': ['mental breakdown', 'struggle when forced to do things', 'wanting to end it', 'trouble sleeping']}


In [ ]:
forth_ten_df.info()
forth_ten_df.to_csv('forth_ten_output.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


## Fifth 10 rows

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
fifth_ten_df = pd.DataFrame(columns=columns)

In [ ]:
fifth_ten_rows =  primate_input_df.iloc[41:50]

total_rows = len(fifth_ten_rows) * 9
progress_bar = tqdm(total=total_rows, desc="Processing Rows")

for index, row in fifth_ten_rows.iterrows():
    post_title = row['post_title']
    post_text = row['post_text']
    for key, value in pre_processed_labels.items():
      boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, value)
      # print(f"The result is: {boolean} and {gpt_answer}")
      if boolean:
        load_gpt_ans_to_json = json.loads(gpt_answer)
        print(load_gpt_ans_to_json)
        result = {
            "bool": boolean,
            "json": load_gpt_ans_to_json
        }

        fifth_ten_df = fifth_ten_df.append({
            'post_title': load_gpt_ans_to_json['post_title'],
            'label': load_gpt_ans_to_json['label'],
            'value': load_gpt_ans_to_json['value']
        }, ignore_index=True)

      if not boolean:
        result = {
            "bool": boolean,
            "error": gpt_answer
        }
      # Add to dataframe
      # Create a new JSON file for each iteration
      file_path = f"gpt_result_{post_title}_{key}.json"  # Customize the file name as desired

      with open(file_path, "w") as json_file:
          json.dump(result, json_file, indent=2)

      progress_bar.update(1)


Processing Rows:   6%|▌         | 5/90 [13:55<3:56:35, 167.01s/it]
<ipython-input-26-8352174ee3c3>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fifth_ten_df = fifth_ten_df.append({

Processing Rows:   1%|          | 1/81 [00:17<23:55, 17.94s/it]

{'label': 'Feeling bad about yourself or that you are a failure or have let yourself or your family down', 'value': 'yes', 'post_title': 'I’ve generally got a good life. Why am I still sad?', 'CLUEs': ['sad', "can't get out of bed", 'no enjoyment in anything', 'lost weight', 'feel unfulfilled', 'nothing matters']}


<ipython-input-26-8352174ee3c3>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fifth_ten_df = fifth_ten_df.append({

Processing Rows:   2%|▏         | 2/81 [00:27<16:58, 12.90s/it]

{'label': 'Feeling down depressed or hopeless', 'value': 'yes', 'post_title': 'I’ve generally got a good life. Why am I still sad?', 'CLUEs': ['sad', "can't get out of bed", 'no enjoyment', "don't like eating", 'lost weight', 'feel completely unfulfilled', 'nothing matters', 'extremely guilty']}


<ipython-input-26-8352174ee3c3>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fifth_ten_df = fifth_ten_df.append({

Processing Rows:   4%|▎         | 3/81 [00:39<16:14, 12.49s/it]

...

{'label': 'Thoughts that you would be better off dead or of hurting yourself in some way', 'value': 'no', 'post_title': 'Rage in my chest', 'CLUEs': ['angry', 'rage feeling']}


<ipython-input-26-8352174ee3c3>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fifth_ten_df = fifth_ten_df.append({

Processing Rows:  99%|█████████▉| 80/81 [55:52<01:13, 73.55s/it] 

{'label': 'Trouble concentrating on things such as reading the newspaper or watching television', 'value': 'no', 'post_title': 'Rage in my chest', 'CLUEs': ['angry', 'rage feeling']}


<ipython-input-26-8352174ee3c3>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fifth_ten_df = fifth_ten_df.append({

Processing Rows: 100%|██████████| 81/81 [56:00<00:00, 53.98s/it]

{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'no', 'post_title': 'Rage in my chest', 'CLUEs': ['Rage', 'filled with rage', 'terrible rage feeling', 'chest']}


In [ ]:
fifth_ten_df.info()
fifth_ten_df.to_csv('fifth_ten_output.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  86 non-null     object
 1   label       86 non-null     object
 2   value       86 non-null     object
dtypes: object(3)
memory usage: 2.1+ KB


# Convert the main file to a flat structure with redundency

In [ ]:
primate_input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003 entries, 0 to 2002
Data columns (total 11 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   post_title                                                                                                                                                             2003 non-null   object 
 1   post_text                                                                                                                                                              2003 non-null   object 
 2   Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down           

In [ ]:
labels = list(primate_input_df.columns[2:])

primate_data_single_label = pd.DataFrame(columns=["post_title", "post_text", "label", "value"])

for index, row in primate_input_df.iterrows():
  for label in labels:
    new_row = {"post_title": row["post_title"],
               "post_text": row["post_text"],
               "label": label,
               "value": row[label]}
    # new_df = new_df.append(new_row, ignore_index=True)
    primate_data_single_label = pd.concat([primate_data_single_label,
                                           pd.DataFrame(new_row, index=[0])],
                                           ignore_index=True)


primate_data_single_label.to_csv("primate_data_single_label.csv", index=False)

In [ ]:
primate_data_single_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18027 entries, 0 to 18026
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  18027 non-null  object
 1   post_text   18027 non-null  object
 2   label       18027 non-null  object
 3   value       18021 non-null  object
dtypes: object(4)
memory usage: 563.5+ KB


In [ ]:
primate_data_single_label.head()

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-bad-about-yourself-or-that-you-are-a-f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-down-depressed-or-hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-tired-or-having-little-energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little-interest-or-pleasure-in-doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving-or-speaking-so-slowly-that-other-people...,0


In [ ]:
primate_data_single_label['label'] = primate_data_single_label['label'].str.replace('-', ' ')

In [ ]:
primate_data_single_label.head(10)

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling bad about yourself or that you are a f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling down depressed or hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling tired or having little energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little interest or pleasure in doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving or speaking so slowly that other people...,0
5,I don't feel original anymore.,"When I was in high school a few years back, I ...",Poor appetite or overeating,0.0
6,I don't feel original anymore.,"When I was in high school a few years back, I ...",Thoughts that you would be better off dead or ...,0.0
7,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble concentrating on things such as readin...,0.0
8,I don't feel original anymore.,"When I was in high school a few years back, I ...",Trouble falling or staying asleep or sleeping ...,0
9,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,Feeling bad about yourself or that you are a f...,1


In [ ]:
primate_data_single_label['label'] = primate_data_single_label['label'].str.strip()

In [ ]:
true_first_90_df = primate_data_single_label[0:90]
true_first_90_df.info()

In [ ]:
true_second_90_df = primate_data_single_label[90:180]
true_second_90_df.info()

In [ ]:
true_third_90_df = primate_data_single_label[180:270]
true_third_90_df.info()

In [ ]:
true_forth_90_df = primate_data_single_label[270:360]
true_forth_90_df.info()

In [ ]:
true_fifth_90_df = primate_data_single_label[360:450]
true_fifth_90_df.info()

## Merging True DFs

In [ ]:
concatenated_true__df = pd.concat([true_first_90_df, true_second_90_df, true_third_90_df, true_forth_90_df, true_fifth_90_df])

In [ ]:
concatenated_true__df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


# Reading and Merging the answers by GPT3.5

In [ ]:
path = '/content/drive/MyDrive/gpt-first-50-records-datasets'

## First 90

In [ ]:
pred_first_90_df = pd.read_csv(f"{path}/first_ten_output_integrated.csv",  encoding='latin-1')
pred_first_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
distinct_values = pred_first_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_first_90_df['value'] = pred_first_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0, 'unknown': 0})

In [ ]:
pred_first_90_df.head(10)

,post_title,label,value
0,I don't feel original anymore.,Feeling bad about yourself or that you are a f...,1
1,I don't feel original anymore.,Feeling down depressed or hopeless,1
2,I don't feel original anymore.,Feeling tired or having little energy,1
3,I don't feel original anymore.,Little interest or pleasure in doing,1
4,I don't feel original anymore.,Moving or speaking so slowly that other people...,0
5,I don't feel original anymore.,Poor appetite or overeating,0
6,I don't feel original anymore.,Thoughts that you would be better off dead or ...,0
7,I don't feel original anymore.,Trouble concentrating on things such as readin...,1
8,I don't feel original anymore.,Trouble falling or staying asleep or sleeping ...,1
9,I dont have anyone to talk to and I dont kno...,Feeling bad about yourself or that you are a f...,1


In [ ]:
pred_first_90_df['post_title'].unique()

array(["I don't feel original anymore.",
       'I don\x92t have anyone to talk to and I don\x92t know what to do anymore',
       'Stuck between moving forward and killing myself',
       'I need help on how to help my girlfriend.',
       "I've been feeling this way for some time", 'I have no motivation',
       'This is me finally talking about \x93it\x94.',
       'Things to say to ppl',
       'Why do I have sudden "bursts" of depression',
       'Being the boyfriend.'], dtype=object)

In [ ]:
true_first_90_df['post_title'].unique()

array(["I don't feel original anymore.",
       'I don’t have anyone to talk to and I don’t know what to do anymore',
       'Stuck between moving forward and killing myself',
       'I need help on how to help my girlfriend.',
       "I've been feeling this way for some tine", 'I have no motivation',
       'This is me finally talking about “it”.', 'Things to say to ppl',
       'Why do I have sudden "bursts" of depression',
       'Being the boyfriend.'], dtype=object)

In [ ]:
true_first_90_df.to_csv('true_first_90.csv', index=False)

In [ ]:
mask = ~(true_first_90_df['post_title'].isin(pred_first_90_df['post_title']) \
         & true_first_90_df['label'].isin(pred_first_90_df['label']))
result = true_first_90_df[mask]

In [ ]:
result

,post_title,post_text,label,value


In [ ]:
label_counts = pred_first_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(pred_first_90_df[pred_first_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

All titles have all nine distinct labels.


## Second 90

In [ ]:
pred_second_90_df = pd.read_csv(f"{path}/seond_ten_output_integrated.csv")
pred_second_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
true_second_90_df['post_title'].unique()

array(['Learning I have Alexithymia really fucked me up.',
       'Angry Issues Causes Everyone to Leave, Should I Be Alone?',
       'I desperately want to feel better.',
       "I'm looking for advice on helping my depressed and, as I've just learned, suicidal grandma.",
       'Help me, please. I have 0 friends in the literal sense. I have been living like this for more than 3 years',
       "Your experience with SSRI's",
       'Is it okay to drop a class for the sake of my own health?',
       "I literally can't do anything right now",
       'Help with insomnia because of depression?',
       'I dont know what to do anymore'], dtype=object)

In [ ]:
pred_second_90_df['post_title'].unique()

array(['Learning I have Alexithymia really fucked me up.',
       'Angry Issues Causes Everyone to Leave, Should I Be Alone?',
       'I desperately want to feel better.',
       "I'm looking for advice on helping my depressed and, as I've just learned, suicidal grandma.",
       'Help me, please. I have 0 friends in the literal sense. I have been living like this for more than 3 years',
       "Your experience with SSRI's",
       'Is it okay to drop a class for the sake of my own health?',
       "I literally can't do anything right now",
       'Help with insomnia because of depression?',
       'I dont know what to do anymore'], dtype=object)

In [ ]:
true_second_90_df.to_csv('true_second_90.csv', index=False)

In [ ]:
distinct_values = pred_second_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_second_90_df['value'] = pred_second_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0})

In [ ]:
pred_second_90_df.head(10)

,post_title,label,value
0,Learning I have Alexithymia really fucked me up.,Feeling bad about yourself or that you are a f...,1
1,Learning I have Alexithymia really fucked me up.,Feeling down depressed or hopeless,1
2,Learning I have Alexithymia really fucked me up.,Feeling tired or having little energy,1
3,Learning I have Alexithymia really fucked me up.,Little interest or pleasure in doing,1
4,Learning I have Alexithymia really fucked me up.,Moving or speaking so slowly that other people...,0
5,Learning I have Alexithymia really fucked me up.,Poor appetite or overeating,0
6,Learning I have Alexithymia really fucked me up.,Thoughts that you would be better off dead or ...,1
7,Learning I have Alexithymia really fucked me up.,Trouble concentrating on things such as readin...,1
8,Learning I have Alexithymia really fucked me up.,Trouble falling or staying asleep or sleeping ...,1
9,"Angry Issues Causes Everyone to Leave, Should ...",Feeling bad about yourself or that you are a f...,1


In [ ]:
mask = ~true_second_90_df['post_title'].isin(pred_second_90_df['post_title'])
result = true_second_90_df[mask]

In [ ]:
mask = ~(true_second_90_df['post_title'].isin(pred_second_90_df['post_title']) \
         & true_second_90_df['label'].isin(pred_second_90_df['label']))
result = true_second_90_df[mask]

In [ ]:
result

,post_title,post_text,label,value


In [ ]:
label_counts = pred_second_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(pred_second_90_df[pred_second_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

Titles missing labels:
Your experience with SSRI's: {'Q2', 'Q8', 'Q1', 'Q3', 'Q6', 'Q7', 'Q9', 'Q4', 'Q5'}


## Third 90

In [ ]:
true_third_90_df['post_title'].unique()

array(["I don't have anything, no future and just.. yeah.",
       'Boyfriend broke up with me a week after I got a shocking diagnosis, says my mood was too bad lately',
       'Life is crumbling to pieces',
       'I don’t know why I’m seeking help here but please just read as much as you can...',
       'Looking into therapy',
       'Admitting no bond with toddler and wondering if its justified or b/c Im just a terrible person.',
       'I need help..', 'Prozac worries', 'So Much Pain In The World',
       "am I suicidal even if I know I won't kill myself?"], dtype=object)

In [ ]:
pred_third_90_df['post_title'].unique()

array(["I don't have anything, no future and just.. yeah.",
       'Boyfriend broke up with me a week after I got a shocking diagnosis, says my mood was too bad lately',
       'Life is crumbling to pieces',
       'I don’t know why I’m seeking help here but please just read as much as you can...',
       'Looking into therapy',
       'Admitting no bond with toddler and wondering if its justified or b/c Im just a terrible person.',
       'I need help..', 'Prozac worries', 'So Much Pain In The World',
       "am I suicidal even if I know I won't kill myself?"], dtype=object)

In [ ]:
pred_third_90_df = pd.read_csv(f"{path}/third_ten_output_integrated.csv")
pred_third_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
distinct_values = pred_third_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_third_90_df['value'] = pred_third_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0})

In [ ]:
pred_third_90_df.head(10)

,post_title,label,value
0,"I don't have anything, no future and just.. yeah.",Feeling bad about yourself or that you are a f...,1
1,"I don't have anything, no future and just.. yeah.",Feeling down depressed or hopeless,1
2,"I don't have anything, no future and just.. yeah.",Feeling tired or having little energy,1
3,"I don't have anything, no future and just.. yeah.",Little interest or pleasure in doing,1
4,"I don't have anything, no future and just.. yeah.",Moving or speaking so slowly that other people...,1
5,"I don't have anything, no future and just.. yeah.",Poor appetite or overeating,1
6,"I don't have anything, no future and just.. yeah.",Thoughts that you would be better off dead or ...,1
7,"I don't have anything, no future and just.. yeah.",Trouble concentrating on things such as readin...,1
8,"I don't have anything, no future and just.. yeah.",Trouble falling or staying asleep or sleeping ...,1
9,Boyfriend broke up with me a week after I got ...,Feeling bad about yourself or that you are a f...,1


In [ ]:
mask = ~(true_third_90_df['post_title'].isin(pred_third_90_df['post_title']) \
         & true_third_90_df['label'].isin(pred_third_90_df['label']))
result = true_third_90_df[mask]

In [ ]:
result

,post_title,post_text,label,value


In [ ]:
label_counts = pred_third_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(pred_third_90_df[pred_third_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

Titles missing labels:
Admitting no bond with toddler and wondering if its justified or b/c Im just a terrible person.: {'Q8', 'Q1', 'Q3', 'Q4', 'Q6', 'Q7', 'Q9', 'Q2', 'Q5'}
Prozac worries: {'Q2', 'Q8', 'Q1', 'Q3', 'Q6', 'Q7', 'Q9', 'Q4', 'Q5'}


In [ ]:
true_third_90_df.to_csv('true_third_90.csv', index=False)

## Forth 90

In [ ]:
true_forth_90_df['post_title'].unique()

array(['i have been diagnosed with depression but my dad refuses to consider medication',
       'why do I feel a void, or a hole in my chest?',
       'Self-Improvement help', 'High functioning depression',
       'Does anyone else feel like the saturation in their life is turned down?',
       "How do I make a female partner who's depressed feel better?",
       'Just need to rant a little bit',
       'Dealing with toxic parents and mental illnesses',
       'My parents sat down with me and said they’ve noticed a change in me, and they are worried about me.',
       'HS ruined my life'], dtype=object)

In [ ]:
pred_forth_90_df['post_title'].unique()

array(['i have been diagnosed with depression but my dad refuses to consider medication',
       'why do I feel a void, or a hole in my chest?',
       'Self-Improvement help', 'High functioning depression',
       'Does anyone else feel like the saturation in their life is turned down?',
       "How do I make a female partner who's depressed feel better?",
       'Just need to rant a little bit',
       'Dealing with toxic parents and mental illnesses',
       'My parents sat down with me and said they’ve noticed a change in me, and they are worried about me.',
       'HS ruined my life'], dtype=object)

In [ ]:
pred_forth_90_df = pd.read_csv(f"{path}/forth_ten_output_integrated.csv")
pred_forth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
distinct_values = pred_forth_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_forth_90_df['value'] = pred_forth_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0})

In [ ]:
pred_forth_90_df.head(10)

,post_title,label,value
0,i have been diagnosed with depression but my d...,Feeling bad about yourself or that you are a f...,1
1,i have been diagnosed with depression but my d...,Feeling down depressed or hopeless,1
2,i have been diagnosed with depression but my d...,Feeling tired or having little energy,1
3,i have been diagnosed with depression but my d...,Little interest or pleasure in doing,1
4,i have been diagnosed with depression but my d...,Moving or speaking so slowly that other people...,1
5,i have been diagnosed with depression but my d...,Poor appetite or overeating,1
6,i have been diagnosed with depression but my d...,Thoughts that you would be better off dead or ...,1
7,i have been diagnosed with depression but my d...,Trouble concentrating on things such as readin...,1
8,i have been diagnosed with depression but my d...,Trouble falling or staying asleep or sleeping ...,0
9,"why do I feel a void, or a hole in my chest?",Feeling bad about yourself or that you are a f...,1


In [ ]:
label_counts = pred_forth_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(pred_forth_90_df[pred_forth_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

All titles have all nine distinct labels.


In [ ]:
true_forth_90_df.to_csv('true_forth_90.csv', index=False)

## Fifth 90

In [ ]:
pred_fifth_90_df['post_title'].unique()

array(['I need some help',
       'I’ve generally got a good life. Why am I still sad?',
       'How can I even tell if antidepressants are working if I have no way to tell?',
       'I need help.', 'I feel like I can’t talk to anyone',
       'Thinking of changing antidepressants',
       'I study at university and things have become EXTREMELY difficult',
       'I need someone',
       "Gf won't get help because she doesn't want to change",
       'Rage in my chest'], dtype=object)

In [ ]:
true_fifth_90_df['post_title'].unique()

array(['I need some help',
       'I’ve generally got a good life. Why am I still sad?',
       'How can I even tell if antidepressants are working if I have no way to tell?',
       'I need help.', 'I feel like I can’t talk to anyone',
       'Thinking of changing antidepressants',
       'I study at university and things have become EXTREMELY difficult',
       'I need someone',
       "Gf won't get help because she doesn't want to change",
       'Rage in my chest'], dtype=object)

In [ ]:
pred_fifth_90_df = pd.read_csv(f"{path}/fifth_ten_output_integrated.csv")
pred_fifth_90_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  90 non-null     object
 1   label       90 non-null     object
 2   value       90 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
distinct_values = pred_fifth_90_df['value'].unique()
distinct_values

array([1, 0])

In [ ]:
pred_fifth_90_df['value'] = pred_fifth_90_df['value'].replace({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0})

In [ ]:
pred_fifth_90_df.head(10)

,post_title,label,value
0,I need some help,Feeling bad about yourself or that you are a f...,1
1,I need some help,Feeling down depressed or hopeless,1
2,I need some help,Feeling tired or having little energy,1
3,I need some help,Little interest or pleasure in doing,1
4,I need some help,Moving or speaking so slowly that other people...,1
5,I’ve generally got a good life. Why am I still...,Feeling bad about yourself or that you are a f...,1
6,I’ve generally got a good life. Why am I still...,Feeling down depressed or hopeless,1
7,I’ve generally got a good life. Why am I still...,Feeling tired or having little energy,1
8,I’ve generally got a good life. Why am I still...,Little interest or pleasure in doing,1
9,I’ve generally got a good life. Why am I still...,Moving or speaking so slowly that other people...,1


In [ ]:
label_counts = pred_fifth_90_df.groupby('post_title')['label'].nunique()

# Identify titles that do not have all nine distinct labels
missing_labels = {}
for title, count in label_counts.items():
    if count < len(pre_processed_labels):
        missing = set(pre_processed_labels.keys()) \
        - set(pred_fifth_90_df[pred_fifth_90_df['post_title'] == title]['label'])
        missing_labels[title] = missing

if not missing_labels:
    print("All titles have all nine distinct labels.")
else:
    print("Titles missing labels:")
    for title, missing in missing_labels.items():
        print(f"{title}: {missing}")

Titles missing labels:
I need some help: {'Q2', 'Q8', 'Q1', 'Q3', 'Q6', 'Q7', 'Q9', 'Q4', 'Q5'}


In [ ]:
true_fifth_90_df.to_csv('true_fifth_90.csv', index=False)

## Merging Pred DFs

In [ ]:
concatenated_pred__df = pd.concat([pred_first_90_df, pred_second_90_df, pred_third_90_df, pred_forth_90_df, pred_fifth_90_df])

In [ ]:
concatenated_pred__df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 89
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   label       450 non-null    object
 2   value       450 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 14.1+ KB


In [ ]:
concatenated_true__df.info()
# concatenated_true__df['value'] = concatenated_true__df['value'].astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_title  450 non-null    object
 1   post_text   450 non-null    object
 2   label       450 non-null    object
 3   value       450 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 14.2+ KB


In [ ]:
# concatenated_pred__df.head()
concatenated_pred__df['value'] = concatenated_pred__df['value'].astype(int)

# Check for the posts which have been faced with error

- Design a new driver code to try the unsuccesful prompting 3 times after facing with exception

In [ ]:
columns = ['post_title', 'label', 'value']

# Create an empty DataFrame
missed_data_df = pd.DataFrame(columns=columns)

In [ ]:
post_text = "When I was 13 years old suffered from bullying at school and I had depression for a long time, my parents knew about it because I put a lot of suicidal stuff on my facebook and I had a friend of my dad on my friends list, so she saw it, talked to me and after she called my parents. They helped me (more my mom) and they took me out of school. \nBut my parents started to say that it was a shame, that everyone would find out about my depression and they wanted to take all of my social medias so no one will know I\u2019m sick. \nAfter that I was scared to talk about them about how I feel. \n\nWhen I was 16 I started to feel empty and scared, I was scare of the night, I was scare to go to sleep, I started to push all my friends away and I stopped liking the things I used to do.\n\nI didn\u2019t know what it was, so I told my parents I was feeling weird, but my dad just told me \u201cYou are just growing up\u201d \nAnd I never talked to them about the topic again.\n\nI\u2019m 18 rn, 2 years hiding my depression and now is killing me, I can\u2019t hide it anymore and I really want to search for help, I want to tell them but I\u2019m scare that they just push me away. \n\nI didn\u2019t told them about my depression because I didn\u2019t wanted to bother them with my problems and I don\u2019t want that they treat me like a bad person. \n\nBtw, sorry for my grammar and I hope some one can help me, I don\u2019t know how to tell my parents about it. "
post_title = "I need some help"

label = pre_processed_labels["Q9"]
boolean, gpt_answer = get_answer_2305_08377(post_title, post_text, label)
# print(f"The result is: {boolean} and {gpt_answer}")
if boolean:
  load_gpt_ans_to_json = json.loads(gpt_answer)
  print(load_gpt_ans_to_json)
  try:
      missed_data_df = missed_data_df.append({
        'post_title': load_gpt_ans_to_json['post_title'],
        'label': load_gpt_ans_to_json['label'],
        'value': load_gpt_ans_to_json['value']
      }, ignore_index=True)
      result = {
        "bool": boolean,
        "json": load_gpt_ans_to_json
      }

  except Exception as e:
      print(str(e)+'\nFirst attempt for getting answer from GPT faild, try 1 to 3 more times with 7 minutes interval...')
      # attempt 1 to 3 more times with 1 minute interval to succeed
      for i in range(3):
          try:
              time.sleep(60)
              print('attempt ' + str(i+1) + ' out of 3')
              missed_data_df = missed_data_df.append({
                'post_title': load_gpt_ans_to_json['post_title'],
                'label': load_gpt_ans_to_json['label'],
                'value': load_gpt_ans_to_json['value']
              }, ignore_index=True)
              result = {
                "bool": boolean,
                "json": load_gpt_ans_to_json
              }
              print('attempt ' + str(i+1) + ' was SUCCESSFUL')
              # break if succeed
              break
          except Exception as e:
              print('attempt ' + str(i + 1) + ' FAILED')
              print(e)

if not boolean:
  result = {
      "bool": boolean,
      "error": gpt_answer
  }



{'label': 'Trouble falling or staying asleep or sleeping too much', 'value': 'yes', 'post_title': 'I need some help', 'CLUEs': ['scared to go to sleep', 'hiding my depression', "can't hide it anymore"]}


<ipython-input-91-93f19c8be76c>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_data_df = missed_data_df.append({


In [ ]:
missed_data_df.head(100)

,post_title,label,value
0,Your experience with SSRI's,Thoughts that you would be better off dead or ...,no
1,Your experience with SSRI's,Trouble concentrating on things such as readin...,yes
2,Your experience with SSRI's,Trouble falling or staying asleep or sleeping ...,no
3,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
4,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
5,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
6,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
7,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
8,Admitting no bond with toddler and wondering i...,Feeling down depressed or hopeless,yes
9,Admitting no bond with toddler and wondering i...,Feeling tired or having little energy,no


In [ ]:
missed_data_df.to_csv('missed_data.csv', index=False)

In [ ]:
# Create a new JSON file for each iteration
file_path = f"gpt_result_{post_title}_{label}.json"  # Customize the file name as desired
try:
  with open(file_path, "w") as json_file:
      json.dump(result, json_file, indent=2)
except Exception as e:
  print(str(e)+'\nFirst attempt for creating file faild, try 1 to 3 more times with 7 minutes interval...')
  # attempt 1 to 3 more times with 1 minute interval to succeed
  for i in range(3):
      try:
          time.sleep(60)
          print('attempt ' + str(i+1) + ' out of 3')
          file_path = f"gpt_result_{post_title}_{label}.json"
          with open(file_path, "w") as json_file:
            json.dump(result, json_file, indent=2)
          print('attempt ' + str(i+1) + ' was SUCCESSFUL')
          # break if succeed
          break
      except Exception as e:
          print('attempt ' + str(i + 1) + ' FAILED')
          print(e)

# Evaluation Metrices for the first 450 records
(actually 50 in the main csv file)

In [ ]:
true_values = concatenated_true__df['value'].values
predicted_values = concatenated_pred__df['value'].values

# Calculate accuracy
accuracy = accuracy_score(true_values, predicted_values)
print(f" ------------------------ The Accuracy of GPT3.5 is: {accuracy} ------------------------")
# Calculate precision
precision = precision_score(true_values, predicted_values)
print(f" ------------------------ The Precision of GPT3.5 is: {precision} ------------------------")

# Calculate recall
recall = recall_score(true_values, predicted_values)
print(f" ------------------------ The Recall of GPT3.5 is: {recall} ------------------------")

# Calculate F1-score
f1 = f1_score(true_values, predicted_values)
print(f" ------------------------ The F1-Score of GPT3.5 is: {f1} ------------------------")


 ------------------------ The Accuracy of GPT3.5 is: 0.5844444444444444 ------------------------
 ------------------------ The Precision of GPT3.5 is: 0.4818181818181818 ------------------------
 ------------------------ The Recall of GPT3.5 is: 0.9085714285714286 ------------------------
 ------------------------ The F1-Score of GPT3.5 is: 0.6297029702970297 ------------------------
